<a href="https://colab.research.google.com/github/samirbehindscreen-jpg/DTSC3020/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [2]:

!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [3]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [ ]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [9]:
# Q1 — Write your answer here
def q1_read_table(html: str) -> pd.DataFrame:
    """
    Return a candidate country-codes table from the HTML.
    Strategy:
      1) read all tables with pandas.read_html
      2) prefer the one that already contains Country/Alpha-2/Alpha-3/Numeric (case-insensitive)
      3) otherwise, pick the widest table that has at least 3 columns
      4) flatten headers before returning
    """
    tables = pd.read_html(html)  # requires lxml installed
    if not tables:
        raise ValueError("No tables were found in the provided HTML.")

    required = {"country", "alpha-2", "alpha-3", "numeric"}
    chosen = None

    for t in tables:
        if t.shape[1] >= 3:
            cols_norm = {str(c).strip().lower() for c in t.columns}
            if required.issubset(cols_norm):
                chosen = t
                break

    if chosen is None:
        # Fallback: pick the widest table with >= 3 columns
        candidates = [t for t in tables if t.shape[1] >= 3]
        if not candidates:
            raise ValueError("No table with >= 3 columns found.")
        chosen = max(candidates, key=lambda d: d.shape[1])

    return flatten_headers(chosen.copy())

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean columns: strip, standardize header names, uppercase Alpha-2/Alpha-3,
    cast Numeric to pandas nullable Int64, and drop clearly invalid rows.
    """
    df = df.copy()

    # 1) Normalize column names (trim)
    df.columns = [str(c).strip() for c in df.columns]

    # 2) Map common variants to canonical names
    colmap = {}
    for c in df.columns:
        cl = c.lower()
        if cl == "country":
            colmap[c] = "Country"
        elif cl in ("alpha-2", "alpha-2 code", "alpha2", "alpha 2"):
            colmap[c] = "Alpha-2"
        elif cl in ("alpha-3", "alpha-3 code", "alpha3", "alpha 3"):
            colmap[c] = "Alpha-3"
        elif cl in ("numeric", "numeric code", "num", "numericcode"):
            colmap[c] = "Numeric"
    df = df.rename(columns=colmap)

    # 3) Keep required cols first (others can remain after them)
    required = ["Country", "Alpha-2", "Alpha-3", "Numeric"]
    ordered_cols = [c for c in required if c in df.columns] + [c for c in df.columns if c not in required]
    df = df[ordered_cols]

    # 4) Trim whitespace in string columns
    for c in df.select_dtypes(include=["object"]).columns:
        df[c] = df[c].astype(str).str.strip()

    # 5) Uppercase Alpha codes (tolerate missing)
    for c in ("Alpha-2", "Alpha-3"):
        if c in df.columns:
            df[c] = df[c].where(df[c].notna(), None)
            df[c] = df[c].astype(str).str.upper().replace({"NAN": pd.NA})

    # 6) Numeric → digits only, then nullable integer
    if "Numeric" in df.columns:
        numeric_clean = df["Numeric"].astype(str).str.extract(r"(\d+)", expand=False)
        df["Numeric"] = pd.to_numeric(numeric_clean, errors="coerce").astype("Int64")

    # 7) Drop rows that are clearly invalid (e.g., missing Country or header echoes)
    if "Country" in df.columns:
        df = df[~df["Country"].astype(str).str.fullmatch(r"\s*Country\s*", case=False, na=False)]
        df = df[df["Country"].notna() & (df["Country"].astype(str).str.len() > 0)]

    # Optional: drop exact duplicates
    df = df.drop_duplicates().reset_index(drop=True)

    return df

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """
    Sort descending by Numeric (dropping NA for the sort) and return Top-N.
    """
    if "Numeric" not in df.columns:
        raise KeyError("Column 'Numeric' is missing; cannot sort.")
    out = df.dropna(subset=["Numeric"]).sort_values("Numeric", ascending=False, kind="mergesort")
    return out.head(top).reset_index(drop=True)
URL = "https://www.iban.com/country-codes"

html = fetch_html(URL)
raw = q1_read_table(html)
clean = q1_clean(raw)

# Save CSV
clean.to_csv("data_q1.csv", index=False, encoding="utf-8")

# Print Top-15 by Numeric (desc)
top15 = q1_sort_top(clean, 15)
print(top15.to_string(index=False))





                                                   Country Alpha-2 Alpha-3  Numeric
                                                    Zambia      ZM     ZMB      894
                                                     Yemen      YE     YEM      887
                                                     Samoa      WS     WSM      882
                                         Wallis and Futuna      WF     WLF      876
                        Venezuela (Bolivarian Republic of)      VE     VEN      862
                                                Uzbekistan      UZ     UZB      860
                                                   Uruguay      UY     URY      858
                                              Burkina Faso      BF     BFA      854
                                     Virgin Islands (U.S.)      VI     VIR      850
                            United States of America (the)      US     USA      840
                              Tanzania, United Republic of      TZ     TZA  

/tmp/ipython-input-167905565.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)  # requires lxml installed


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [ ]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [24]:
from urllib.parse import urljoin
import re
from bs4 import BeautifulSoup
import pandas as pd   # <— make sure this import is present too

HN_BASE = "https://news.ycombinator.com/"

def _parse_int(text: str) -> int:
    """Return the first integer found in text; if none, 0."""
    if text is None:
        return 0
    m = re.search(r"(\d+)", str(text))
    return int(m.group(1)) if m else 0


def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional)."""
    soup = BeautifulSoup(html, "lxml")
    items = []

    for row in soup.select("tr.athing"):
        rank_txt = row.select_one("span.rank")
        rank = _parse_int(rank_txt.get_text(strip=True) if rank_txt else None)

        a = row.select_one(".titleline a") or row.select_one("a.storylink")
        title = a.get_text(strip=True) if a else ""
        href = a.get("href", "") if a else ""
        link = urljoin(HN_BASE, href) if href else ""

        sub = row.find_next_sibling("tr")
        subtext = sub.select_one(".subtext") if sub else None

        # Safely get points_txt, handling cases where 'span.score' might not be found
        points_element = subtext.select_one("span.score") if subtext else None
        points_txt = points_element.get_text(strip=True) if points_element else None
        points = _parse_int(points_txt)

        u = subtext.select_one("a.hnuser") if subtext else None
        user = u.get_text(strip=True) if u else ""

        comments = 0
        if subtext:
            for a2 in reversed(subtext.select("a")):
                t = a2.get_text(strip=True).lower()
                if "comment" in t or "discuss" in t:
                    comments = _parse_int(t)
                    break

        items.append({
            "rank": rank,
            "title": title or "",
            "link": link or "",
            "points": points,
            "comments": comments,
            "user": user or ""
        })

    return pd.DataFrame(items, columns=["rank", "title", "link", "points", "comments", "user"])


def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values."""
    df = df.copy()

    for c in ("title", "link", "user"):
        if c in df.columns:
            df[c] = df[c].fillna("").astype(str).str.strip()

    for c in ("rank", "points", "comments"):
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

    if "title" in df.columns and "link" in df.columns:
        df = df[(df["title"] != "") | (df["link"] != "")]
    return df.reset_index(drop=True)


def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N."""
    if "points" not in df.columns:
        raise KeyError("Column 'points' missing; cannot sort.")
    return df.sort_values("points", ascending=False, kind="mergesort").head(top).reset_index(drop=True)


# === Run ===
URL = "https://news.ycombinator.com/"
html = fetch_html(URL)
raw = q2_parse_items(html)
clean = q2_clean(raw)

clean.to_csv("data_q2.csv", index=False, encoding="utf-8")

top15 = q2_sort_top(clean, 15)
print(top15.to_string(index=False))


 rank                                                                         title                                                                                                   link  points  comments         user
    9                                                      Leaving Meta and PyTorch                                    https://soumith.ch/blog/2025-11-06-leaving-meta-and-pytorch.md.html     652       158     saikatsg
   30                            Meta projected 10% of 2024 revenue came from scams https://sherwood.news/tech/meta-projected-10-of-2024-revenue-came-from-scams-and-banned-goods-reuters/     565       438      donohoe
   11                                                               A Fond Farewell                                      https://www.farmersalmanac.com/fond-farewell-from-farmers-almanac     543       196       erhuve
   23       Rockstar employee shares account of the company's union-busting efforts                              https://gtaforu